In [7]:
#I am setting this environment to practice and validate dataset and build something out of it, hopefully!!
#and for easier analaysis, ofcourse, VSCode is not Data Engineers friendly, HAHAHA!

In [8]:
import pandas as pd
import os
import datetime

In [9]:
staging_directory="../staging/"
bulk_csv_list=[ x for x in os.listdir(staging_directory) if 'bulk' in x ]
bulk_df_list=[pd.read_csv(staging_directory+file) for file in bulk_csv_list]
bulk_df=pd.concat(bulk_df_list, axis=0, ignore_index=True)

bulk_df.head()

,Date,Symbol,Security Name,Client Name,Buy/Sell,Quantity Traded,Trade Price / Wght. Avg. Price,Remarks
0,28-AUG-2023,ABCOTS,A B Cotspin India Limited,RAJ KUMAR JINDAL,BUY,68000,94.30,-
1,28-AUG-2023,AGUL,A G Universal Limited,PRAKASH RATANCHAND SHAH,BUY,36000,63.06,-
2,28-AUG-2023,AGUL,A G Universal Limited,YUGA STOCKS AND COMMODITIES PRIVATE LIMITED .,BUY,28000,64.72,-
3,28-AUG-2023,ANTGRAPHIC,Antarctica Graphics Ltd,MULTIPLIER SHARE & STOCK ADVISORS PRIVATE LIMITED,BUY,1000000,0.87,-
4,28-AUG-2023,ATALREAL,Atal Realtech Limited,INDRANIL NANDI,BUY,350000,137.94,-


In [10]:
#bulk Df is basically block deals happend by the client.
#These clients can be FII and DII. We will build something to identfy that later on.
#for now, I am trying to consolidate the client regime, how much of Indian stock they bought.
staging_directory="../staging/"
bulk_csv_list=[ x for x in os.listdir(staging_directory) if 'bulk' in x ]
bulk_df_list=[pd.read_csv(staging_directory+file) for file in bulk_csv_list]
bulk_df=pd.concat(bulk_df_list, axis=0, ignore_index=True)

#Schema changes: I am changing the schema as it is not good way to keep spaces in between.
#let's create a standard of camel casing i.e Quantity Traded -> QuantityTraded.
#Also, Let's Fix the Date Standards. 25-Aug-2023 -> 2023-08-25

new_schema={'Symbol':'SecuritiesCode','Client Name':'ClientName','Quantity Traded':'QuantityTraded', 'Trade Price / Wght. Avg. Price':'MeanWghtPrice', 'Buy/Sell':'Transaction'}
bulk_df_cleaned=bulk_df.rename(columns=new_schema).drop(columns=['Security Name','Remarks']).drop_duplicates()
bulk_df_cleaned['Date']=pd.to_datetime(bulk_df_cleaned.Date).dt.strftime('%Y-%m-%d')

#Negating QuantityTraded for Sell Transaction type to get the net Transaction.
#Net QunatityTraded
# - Positive -> Buy
# - Negative -> Sell
def negate_quantityTraded(Transaction,QuantityTraded):
    if Transaction == "SELL":
        return -1*QuantityTraded
    else:
        return QuantityTraded

bulk_df_cleaned['QuantityTraded']=bulk_df_cleaned.apply(lambda x: negate_quantityTraded(x['Transaction'],x['QuantityTraded']),axis=1)
bulk_df_cleaned.sort_values(by=['ClientName'],ascending=False).head(20)

#Possible case:
#   -  Now, we don't know how much shares these Client posses. Anyways we are concerned with Change.
#   -  There could be case of Intraday Trading i.e Buyer will buy/sell or sell/buy in same day, net buy=0
#   -  long term i.e CNC buy.

,Date,SecuritiesCode,ClientName,Transaction,QuantityTraded,MeanWghtPrice
86,2023-08-28,ZEAL,YUGA STOCKS AND COMMODITIES PRIVATE LIMITED .,SELL,-94800,228.28
2,2023-08-28,AGUL,YUGA STOCKS AND COMMODITIES PRIVATE LIMITED .,BUY,28000,64.72
41,2023-08-28,AGUL,YUGA STOCKS AND COMMODITIES PRIVATE LIMITED .,SELL,-24000,64.73
38,2023-08-28,ZEAL,YUGA STOCKS AND COMMODITIES PRIVATE LIMITED .,BUY,94800,226.91
82,2023-08-28,VIKASECO,VISHWAS FINCAP SERVICES PRIVATE LIMITED,SELL,-8036285,3.02
35,2023-08-28,VIKASECO,VISHWAS FINCAP SERVICES PRIVATE LIMITED,BUY,8636285,2.98
37,2023-08-28,VISASTEEL,VISA INDUSTRIES LIMITED,BUY,2200000,14.30
50,2023-08-28,CREST,VERNALIS CAPITAL PVT LTD,SELL,-150000,237.00
66,2023-08-28,MORARJEE,SUNITA GUPTA,SELL,-192022,21.15
65,2023-08-28,MEGAFLEX,SHITU GUPTA,SELL,-90000,50.40


In [11]:
#now that we have some cleaning done, let check the regime of Client.
#Regime of Client: Varied Stocks that single client has bought.
#PS: you will see so many reassignments, coz I hate using more variables.

ClientPortfolio=bulk_df_cleaned
tdf=ClientPortfolio.groupby(['ClientName','Date','SecuritiesCode'])[['QuantityTraded']].sum().reset_index()

#TradeType: Indicates type of Trade occurred depending on Net QuantityTraded.
#if Net QunatityTraded is 0 -> Intraday Trading else or else we will consider it long term.
def determine_trade_type(NetQunatityTrade) -> str :
    if NetQunatityTrade == 0:
        return "INTRADAY"
    elif NetQunatityTrade > 0:
        return "BUY"
    else :
        return "SELL"

tdf['TradeType']=tdf.apply(lambda x: determine_trade_type(x['QuantityTraded']), axis=1 )
#Filter Intraday data
tdf_Intraday=tdf[ tdf['TradeType']=='INTRADAY']
tdf_other=tdf[ tdf['TradeType']!='INTRADAY']

#TODO:reconcile outputdataset.

tdf_Intraday.to_csv("../data/IntradayData_20230828.csv")
tdf_other.to_csv("../data/ClientPortFolio_20230828.csv")

#I am debating wether to filter out Intraday TradeType for now or not?
#But Keeping might give us better insight on which Stocks are good for intraday

tdf_other.head(20)
#I know this will only give us the Client who bought most number of stocks in a day, which it bought that is still with bulk_df_cleaned
#I have alogorithm to store all the stocks bought in fixed lenght, will implement it later on. 
# Just for guess, it has to do with bit map. HAHA!


,ClientName,Date,SecuritiesCode,QuantityTraded,TradeType
2,ABARC-AST-002-TRUST,2023-08-28,RTNPOWER,-44095395,SELL
3,ACHINTYA SECURITIES PRIVATE LIMITED,2023-08-28,TRU,-900197,SELL
4,AJAY SALVI,2023-08-28,SAHANA,-80000,SELL
5,ASSETS CARE AND RECONSTRUCTION ENTERPRISE LIMITED,2023-08-28,WALCHANNAG,-573421,SELL
6,BHARTI JIKESH SHAH,2023-08-28,SONAHISONA,125000,BUY
7,CITADEL SECURITIES INDIA MARKETS PRIVATE LIMITED,2023-08-28,RAILTEL,1158057,BUY
8,ELIXIR WEALTH MANAGEMENT PRIVATE LIMITED,2023-08-28,SAHANA,3000,BUY
9,G G ENGINEERING LIMITED,2023-08-28,GISOLUTION,-335687,SELL
10,GARAPATI RADHAKRISHNA,2023-08-28,RKEC,-145000,SELL
11,GAZANIA ADVISORY LLP,2023-08-28,63MOONS,-300000,SELL
